In [1]:
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm, trange
from torch import nn

In [2]:
import torch
from transformers import BertModel

if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
    torch.cuda.empty_cache()
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'

Current device: GeForce GTX 960M


In [3]:
from utils.Constants import tokenizer, PRE_TRAINED_MODEL_NAME
from dataLoader.DataLoader import get_data_loader

max_test_size = 100
test_data_loader = get_data_loader(data_type = 'test', tokenizer = tokenizer, batch_size = 1, max_size=max_test_size)

100%|██████████| 100/100 [00:00<00:00, 282.62it/s]


We have 15878 test data with 5230 tables.


In [4]:
from models.QABert import QABertTrainer
from models.SelectRanker import SelectRankerTrainer
from models.WhereRanker import WhereRankerTrainer
from models.WhereConditionClassifier import WhereConditionClassifierTrainer
from models.WhereNumberClassifier import WhereNumberClassifierTrainer
from models.AggregationClassifier import AggregationClassifierTrainer

from train.Trainer import load_model

bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
for param in bert.parameters():
    param.requires_grad = False

path=None

if path:
    models = load_model(path, test_data_loader, device)
else:
    models = dict(
        selection_trainer = SelectRankerTrainer(device, test_data_loader, bert),
        agg_class_trainer = AggregationClassifierTrainer(device, test_data_loader, bert, use_pretrained=False),
        where_ranker_trainer = WhereRankerTrainer(device, test_data_loader, bert),
        where_cond_class_trainer = WhereConditionClassifierTrainer(device, test_data_loader, bert, use_pretrained=False),
        where_numb_class_trainer = WhereNumberClassifierTrainer(device, test_data_loader, bert),
        qa_trainer = QABertTrainer(device, test_data_loader, bert, use_pretrained=False),
    )

In [5]:
from train.Trainer import get_eval

eval = get_eval(models, test_data_loader, max_test_size, device)

print(eval)


Eval:   0%|          | 0/100 [00:00<?, ?it/s]

c:\src\python\dke\um_anlp_project\seqtosql\venv\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\src\python\dke\um_anlp_project\seqtosql\venv\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{'selection_trainer': {'loss': 1.76735356092453, 'acc': 0.38}, 'agg_class_trainer': {'loss': 2.1663984405994414, 'acc': 0.0}, 'where_ranker_trainer': {'loss': 0.8684695595502854, 'acc': 0.2}, 'where_cond_class_trainer': {'loss': nan, 'acc': nan}, 'where_numb_class_trainer': {'loss': 1.9015950155258179, 'acc': 0.11}, 'qa_trainer': {'loss': nan, 'acc': nan}}
